In [5]:
import pandas as pd

In [2]:
!unrar e "/content/drive/MyDrive/Walmark/walmart.rar"


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/drive/MyDrive/Walmark/walmart.rar

Extracting  features.csv                                                   4%  OK 
Extracting  sampleSubmission.csv                                          10%  OK 
Extracting  stores.csv                                                    10%  OK 
Extracting  test.csv                                                      16%  OK 
Extracting  train.csv                                                     99%  OK 
All OK


In [13]:
import joblib
load_model = joblib.load("/content/drive/MyDrive/Walmark/walmark_sales.sav")

In [6]:
data = pd.read_csv("/content/test.csv", sep=',',header=0)
features = pd.read_csv("/content/features.csv", sep=',',header=0)
stores = pd.read_csv("/content/stores.csv", sep=',',header=0)
data = data.merge(stores, how='left').merge(features, how='left')

In [7]:
data['Month'] = pd.to_datetime(data['Date']).dt.month
data['Year'] = pd.to_datetime(data['Date']).dt.year
data['Week'] = pd.to_datetime(data['Date']).dt.isocalendar().week

In [10]:
data[['MarkDown1','MarkDown2','MarkDown3','MarkDown4', 'MarkDown5']] = data[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].fillna(0)
data = data.fillna(0)
column_date = data['Date']
data['Month'] = pd.to_datetime(data['Date']).dt.month
data = data.drop(columns=["Date",'Fuel_Price', 'Unemployment'])
data

,Store,Dept,IsHoliday,Size,Temperature,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Month,Year,Week,Type_A,Type_B,Type_C
0,1,1,False,151315,55.32,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,11,2012,44,1,0,0
1,1,1,False,151315,61.24,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,11,2012,45,1,0,0
2,1,1,False,151315,52.92,9696.28,292.10,103.78,1133.15,6612.69,223.512911,11,2012,46,1,0,0
3,1,1,True,151315,56.23,883.59,4.17,74910.32,209.91,303.32,223.561947,11,2012,47,1,0,0
4,1,1,False,151315,52.34,2460.03,0.00,3838.35,150.57,6966.34,223.610984,11,2012,48,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115059,45,98,False,118221,76.05,4842.29,975.03,3.00,2449.97,3169.69,0.000000,6,2013,26,0,1,0
115060,45,98,False,118221,77.50,9090.48,2268.58,582.74,5797.47,1514.93,0.000000,7,2013,27,0,1,0
115061,45,98,False,118221,79.37,3789.94,1827.31,85.72,744.84,2150.36,0.000000,7,2013,28,0,1,0
115062,45,98,False,118221,82.84,2961.49,1047.07,204.19,363.00,1059.46,0.000000,7,2013,29,0,1,0


In [14]:
predict = load_model.predict(data)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.6s finished


In [15]:
data['Weekly_Sales'] = predict
data['Date'] = column_date
data['Id'] = data['Store'].astype(str) + '_' +  data['Dept'].astype(str) + '_' +  data['Date'].astype(str)
data = data[['Id', 'Weekly_Sales']]

In [16]:
data.to_csv('/content/drive/MyDrive/Walmark/dataset_output.csv', index=False)